[View in Colaboratory](https://colab.research.google.com/github/mzhang32/nanohub-project/blob/master/first_submodel.ipynb)

#First Submodel



###Given a set of inputs about angle distribution and nanotube length and filament size, generate some line segments .
Is this where we use Monte Carlo? I dunno

In [0]:
# TODO

 ### Graphically visualize the model.
 We may have to import some libraries.

In [0]:
# TODO

###Represent the distribution of the nanowires and the distances between them as a graph.
Like what this paper has done: [Computer Simulation of Field-Controlled Percolation
in 3D System of Straight Nanotubes](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8365699&tag=1)

In [0]:
# TODO

### Use the graph to compute a conductive path.
Graph theory is great. Details are in the working.

In [0]:
# TODO

###Graphically visualize the model with the path.

In [0]:
# TODO